<a href="https://colab.research.google.com/github/ColeDCrawford/mp-scrape/blob/master/MP_Routes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re

import pprint
pp = pprint.PrettyPrinter(indent=2)

idPattern = re.compile("^[0-9]{1,10}$")
uniqueRouteIds = set()

import pandas as pd
import time

import csv

# from google.colab import files
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
apikey = ''
all_routes = []
# pre = '/content/gdrive/My Drive/Mountain Project Scrape/'
pre = ""

route_location_ids = {}
with open(f"{pre}route_loc_ids.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    route_location_ids[int(row['routeId'])] = int(row['locationId'])

with open(f"{pre}route_loc_ids-after-indiana.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    route_location_ids[int(row['routeId'])] = int(row['locationId'])

In [ ]:
def get_routes(ids, pause=5):
  time.sleep(pause)
  str_ids = ','.join(str(x) for x in ids)
  params = {
      'key': apikey,
      'routeIds': str_ids
  }
  r = requests.get('https://www.mountainproject.com/data/get-routes', params=params)
  if(r.json()['success'] == 1):
    routes = r.json()['routes']
    # enhance with location ID
    for route in routes:
      locationId = route_location_ids[route['id']]
      route['locationId'] = locationId
    return routes
  else:
    print("Error")
    print(r.json())
    return False

In [ ]:
routeIds = list(route_location_ids.keys())
print(f"Total route IDs: {len(routeIds)}")

def divide_chunks(l, n): 
  for i in range(0, len(l), n):  
    yield l[i:i + n]

finished = set()
with open(f"{pre}route_data_v2.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open(f"{pre}route_data_v1.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open(f"{pre}route_data_v3.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open(f"{pre}route_data_v4.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open(f"{pre}route_data_v5.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open(f"{pre}route_data_v6.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
print(f"Total finished: {len(finished)}")

toProcess = []
for i in routeIds:
  if(i not in finished):
    toProcess.append(i)

print(f"To process: {len(toProcess)}")

In [ ]:
chunked = list(divide_chunks(toProcess, 200))
for num, chunk in enumerate(chunked):
  # print(chunk)
  routes = get_routes(chunk)
  last = routes[-1]['id']
  all_routes.extend(routes)
  if((num % 10 == 0) or (num == len(chunked)-1)):
    # write
    df = pd.DataFrame(all_routes)
    df.to_csv(f"{pre}route_data_v6.csv", index=False)
    print(f"Previous runs: {len(finished)} | As of last run: {num*200} | Remaining: {len(toProcess) - (num*200)}")

In [ ]:
print(len(all_routes))

In [ ]:
joined = []
for i in range(1,6):
    print(i)
    with open(f"route_data_v{i}.csv") as read_obj:
      csv_dict_reader = csv.DictReader(read_obj)
      for row in csv_dict_reader:
           joined.append(row)

In [ ]:
df_export = pd.DataFrame(joined)
df_export.to_csv("mp_routes_joined.csv", index=False)